# **Eventure : Category Classifier**

Determining categories based on user input using Natural Language Processing

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
!python --version
print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.6 MB/s eta 0:00:00
Python 3.10.12
2.15.0


In [2]:
# Membaca file excel
file_path = '/content/drive/MyDrive/Capstone Project/datasets/data training.csv'
df = pd.read_csv(file_path)

# Pastikan kolom 'text' dan 'label' ada di DataFrame
if 'text' not in df.columns or 'label' not in df.columns:
    raise ValueError("File Excel harus memiliki kolom 'text' dan 'label'")

df.head(500)

,text,label,all_category,sum_category
0,ke tempat yang menyegarkan,wisata,wisata,19.0
1,berkeliling naik ATV,wisata,edukasi,23.0
2,mendaki gunung di alam terbuka,wisata,olahraga,22.0
3,melewati hutan yang sangat luas dan liar,wisata,hiburan,14.0
4,piknik di taman,wisata,bisnis,26.0
...,...,...,...,...
140,menonton kompetisi tari,budaya,NaN,NaN
141,membeli barang-barang khas daerah,budaya,NaN,NaN
142,budaya,budaya,NaN,NaN
143,mengunjungi pasar tradisional,budaya,NaN,NaN


In [3]:
# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk stemming teks
def stem_text(text):
    return stemmer.stem(text)

# Terapkan stemming pada kolom 'text'
df['text'] = df['text'].apply(stem_text)

df['text'].head(500)

0                       ke tempat yang segar
1                          keliling naik atv
2                   daki gunung di alam buka
3      lewat hutan yang sangat luas dan liar
4                            piknik di taman
                       ...                  
140                    tonton kompetisi tari
141                  beli barang khas daerah
142                                   budaya
143                 unjung pasar tradisional
144                                     lain
Name: text, Length: 145, dtype: object

In [4]:
# Inisialisasi tokenizer
tokenizer = Tokenizer(num_words=500, oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'])

# Konversi teks menjadi sequence
sequences = tokenizer.texts_to_sequences(df['text'])
padded_sequences = pad_sequences(sequences, padding='post')

# Konversi label menjadi angka
label_map = {
    'wisata': 0,
    'edukasi': 1,
    'olahraga': 2,
    'hiburan': 3,
    'musik': 4,
    'bisnis': 5,
    'budaya': 6,
    'lainnya': 7
}
labels = df['label'].map(label_map).values

# One-hot encoding pada label
labels_one_hot = to_categorical(labels)

In [5]:
# Definisikan model
model = Sequential([
    Embedding(input_dim=500, output_dim=64, input_length=padded_sequences.shape[1]),
    LSTM(64),
    Dense(8, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 64)            32000     
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
Total params: 65544 (256.03 KB)
Trainable params: 65544 (256.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(
    padded_sequences,
    labels_one_hot,
    epochs=200
)

Epoch 1/200
5/5 [==============================] - 3s 29ms/step - loss: 2.0824 - accuracy: 0.1034
Epoch 2/200
5/5 [==============================] - 0s 21ms/step - loss: 2.0638 - accuracy: 0.1586
Epoch 3/200
5/5 [==============================] - 0s 39ms/step - loss: 2.0471 - accuracy: 0.1586
Epoch 4/200
5/5 [==============================] - 0s 31ms/step - loss: 2.0251 - accuracy: 0.1655
Epoch 5/200
5/5 [==============================] - 0s 34ms/step - loss: 1.9991 - accuracy: 0.1655
Epoch 6/200
5/5 [==============================] - 0s 61ms/step - loss: 1.9600 - accuracy: 0.1724
Epoch 7/200
5/5 [==============================] - 0s 44ms/step - loss: 1.9089 - accuracy: 0.2000
Epoch 8/200
5/5 [==============================] - 0s 23ms/step - loss: 1.8379 - accuracy: 0.2690
Epoch 9/200
5/5 [==============================] - 0s 24ms/step - loss: 1.7145 - accuracy: 0.3310
Epoch 10/200
5/5 [==============================] - 0s 25ms/step - loss: 1.5274 - accuracy: 0.3655
Epoch 11/200
5/5 [=

In [7]:
# Save the model
model.save('model-nlp-with-stem.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
def predict(text):
    # Stemming teks input
    stemmed_text = stem_text(text)

    # Contoh prediksi
    new_text = [stemmed_text]
    new_seq = tokenizer.texts_to_sequences(new_text)
    new_padded = pad_sequences(new_seq, maxlen=padded_sequences.shape[1], padding='post')

    # Prediksi
    prediction = model.predict(new_padded)
    predicted_label_index = prediction.argmax(axis=-1)
    predicted_label = list(label_map.keys())[predicted_label_index[0]]

    # Menampilkan semua probabilitas kategori
    category_probabilities = {category: prob for category, prob in zip(label_map.keys(), prediction[0])}

    print(f'\nTeks: "{new_text[0]}" diklasifikasikan sebagai: {predicted_label}')
    print('\nProbabilitas kategori:')
    for category, probability in category_probabilities.items():
        print(f'{category}: {probability:.4f}')

In [9]:
predict('kegiatan yang mengurangi kadar kalori')

1/1 [==============================] - 1s 504ms/step

Teks: "giat yang kurang kadar kalori" diklasifikasikan sebagai: olahraga

Probabilitas kategori:
wisata: 0.0007
edukasi: 0.0007
olahraga: 0.9975
hiburan: 0.0001
musik: 0.0001
bisnis: 0.0001
budaya: 0.0008
lainnya: 0.0000
